<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_Expression_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain的新特性 - Expression Language

`LangChain Expression Language` 是一种以声明式方法，轻松地将链或组件组合在一起的机制。通过利用管道操作符，构建的任何链将自动具有完整的同步、异步和流式支持。

## Python管道 - Pipe

`Python` 的 `Pipe` 提供了管道实现。请参考 [https://github.com/JulienPalard/Pipe](https://github.com/JulienPalard/Pipe)。 来看几个例子

### 安装

In [1]:
# !pip install -U pipe

### 最简单一个例子

In [3]:
from pipe import select, where

numbers = [1, 2, 3, 4, 5]
result = list(numbers | where(lambda x: x % 2 == 0) | select(lambda x: x * 2))

result

[4, 8]

### 小小进阶

自定义管道 `uppercase` - 接受一个 `iterable` 参数

In [5]:
from pipe import Pipe

uppercase = Pipe(lambda iterable: (x.lower() for x in iterable))

words = ['red', 'green', 'blue', 'YELLOW']

uppercase_words = list(words | uppercase)

uppercase_words

['red', 'green', 'blue', 'yellow']

## LangChain Expression Language与管道

`LEL` 通过管道定义操作序列，帮助程序员以更加优雅简洁的编码方式构建功能逻辑。我们来看看如何通过表达式来重构几个经典的LangChain实例。

### 安装

我们需要安装最新版本的 `langchain` 以确保具有 `LEL` 功能的支持。

In [23]:
# !pip install -q -U langchain openai

### 提示词模版与模型


In [24]:
# import os

# os.environ['OPENAI_API_KEY'] = '您的有效openai api key'

In [11]:
# from langchain.llms import ChatGLM
# from langchain import PromptTemplate, LLMChain

# template = """{question}"""  
# prompt = PromptTemplate(template=template, input_variables=["question"])

# # llm = ChatGLM(endpoint_url="http://localhost:8000") 
# llm = ChatGLM(endpoint_url="http://10.26.128.38:8595")


# llm_chain = LLMChain(prompt=prompt, llm=llm)

# question = "What is the capital of China?"
# llm_chain.run(question)


'''
Author: DingWang wangding19@mails.ucas.ac.cn
Date: 2023-08-04 09:52:47
LastEditors: DingWang wangding19@mails.ucas.ac.cn
LastEditTime: 2023-08-04 11:37:16
FilePath: /ChatGLM2-6B/LLMForLangChain.py
Description: 

Copyright (c) 2023 by ${git_name_email}, All Rights Reserved. 
'''
import time
import logging
import requests
from typing import Optional, List, Dict, Mapping, Any

import langchain
from langchain.llms.base import LLM
from langchain.cache import InMemoryCache

logging.basicConfig(level=logging.INFO)
# 启动llm的缓存
langchain.llm_cache = InMemoryCache()

class ChatGLM(LLM):
    
    # 模型服务url
    url = "http://10.26.128.38:8595/chat"

    @property
    def _llm_type(self) -> str:
        return "chatglm"

    def _construct_query(self, prompt: str) -> Dict:
        """构造请求体
        """
        query = {
            "human_input": prompt
        }
        return query

    @classmethod
    def _post(cls, url: str,
        query: Dict) -> Any:
        """POST请求
        """
        _headers = {"Content_Type": "application/json"}
        with requests.session() as sess:
            resp = sess.post(url, 
                json=query, 
                headers=_headers, 
                timeout=60)
        return resp

    
    def _call(self, prompt: str, 
        stop: Optional[List[str]] = None) -> str:
        """_call
        """
        # construct query
        query = self._construct_query(prompt=prompt)

        # post
        resp = self._post(url=self.url,
            query=query)
        
        if resp.status_code == 200:
            resp_json = resp.json()
            predictions = resp_json["response"]
            return predictions
        else:
            return f"请求模型 in {resp.status_code}" 
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters.
        """
        _param_dict = {
            "url": self.url
        }
        return _param_dict

# if __name__ == "__main__":
    
#     # import 
    
#     llm = ChatGLM()
#     while True:
#         human_input = input("Human: ")

#         begin_time = time.time() * 1000
#         # 请求模型
#         response = llm(human_input, stop=["you"])
#         end_time = time.time() * 1000
#         used_time = round(end_time - begin_time, 3)
#         logging.info(f"chatGLM process time: {used_time}ms")

#         print(f"ChatGLM: {response}")

#### 提示词模板与模型的传统用法

In [12]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage
)
from langchain.chains import LLMChain


# llm = 

human_template="Show me the HEX code of color {color_name}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
chain = LLMChain(llm=ChatGLM(), prompt=chat_prompt)

chain.run("RED")

'The HEX code of color red is: #FF0000.'

#### 通过 `LEL` 连接提示词模板与模型

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

model = ChatGLM()
prompt = ChatPromptTemplate.from_template("Show me the HEX code of color {color_name}")

chain = prompt | model

chain.invoke({"color_name": "RED"})

'The HEX code of color red is: #FF0000.'

### 一个稍稍复杂的例子

现在我们给刚才搭建的管道追加一些环节

#### 添加标准输出解析

In [17]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt | model | StrOutputParser()

chain.invoke({"color_name": "RED"})

'The HEX code of color red is: #FF0000.'

#### 添加函数调用

我们来给管道中的模型添加一些函数调用。注，我们并不真正调用函数，只解析出函数调用的数据。

`JsonOutputFunctionsParser` 用来将函数调用的回复解析为JSON格式，请参考[API 文档](https://api.python.langchain.com/en/latest/output_parsers/langchain.output_parsers.openai_functions.JsonKeyOutputFunctionsParser.html)

In [16]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

functions = [
    {
      "name": "save_color_code",
      "description": "Save the HEX code of color and its name",
      "parameters": {
        "type": "object",
        "properties": {
          "hex_code": {
            "type": "string",
            "description": "The HEX code of the color"
          },
          "color": {
            "type": "string",
            "description": "The color name"
          }
        },
        "required": ["hex_code", "color"]
      }
    }
  ]
chain = prompt | model.bind(function_call = {"name": "save_color_code"}, functions = functions) | JsonOutputFunctionsParser()

chain.invoke({"color_name": "RED"})

OutputParserException: This output parser can only be used with a chat generation.